In [2]:
import pandas as pd
import numpy as np
import urllib.request
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pandas as pd
import glob
import os

from konlpy.tag import Okt
okt=Okt()

In [3]:
data = pd.read_csv("국내여행지_크롤링.csv", encoding='utf-8')

In [5]:
data

,제목,링크,내용,country,city
0,친구가족들과 1박2일 <b>가평여행<\/b>:D (프롬밀크 카페...,https://blog.naver.com/emily_0217/223020320907,2월 초에 다녀온 세가족 가평 1박2일 기록해보려한다.\n남편 친구네 가족이지만 나...,경기,가평
1,경기도 가볼만한곳 <b>가평 여행<\/b> 관광지 동물체험 가평 양떼목장...,https://blog.naver.com/tcacyc/222979562087,경기도 가볼만한곳 가평 여행 관광지 동물체험 가평 양떼목장 동물원\n서울 근교에 있...,경기,가평
2,"청량리역 ktx, itx 카페, 김밥 등 feat. <b>가평여행<\/b>",https://blog.naver.com/pinkplum80/222975800122,"가벼운 남이섬-가평 여행을\n시작하는 날,\n일찌감치 청량리역 ktx, itx 에 ...",경기,가평
3,<b>가평여행<\/b> 설악막국수 춘천닭갈비 먹고왔어요,https://blog.naver.com/coom91/223039250726,50m\n© NAVER Corp.\n설악막국수춘천닭갈비 서관\n경기도 가평군 설악면...,경기,가평
4,[<b>가평여행<\/b> 장보기] 마트 물가비교1. 조은마트(픽업) vs 와캠핑...,https://blog.naver.com/gud_log/223051140444,오늘은 요새 제가 가평여행을\n한달에 2~3번 가면서 잇님들께 도움될만한 정보가\n...,경기,가평
...,...,...,...,...,...
82502,<b>횡성 여행<\/b>｜리포터 <b>추천<\/b> 맛집! <b>횡성<\/b>더덕 ...,https://blog.naver.com/himawari_tw/222044996596,강\n원도 횡성에서 유명한 것이 바로 '더덕'이죠. 횡성더덕으로 코스요리를 전문으로...,강원,횡성
82503,특실 #<b>횡성<\/b>버스터미널 #<b>횡성<\/b>호텔 <b>추천<\/b> #...,https://blog.naver.com/tmdwns880908/223058247691,횡성 호텔#호텔 타임 #특실 #횡성 버스터미널 #횡성 호텔 추천 #금연 호텔\n강원...,강원,횡성
82504,<b>여행<\/b>\/ <b>횡성<\/b> 살구나무 글램핑 <b>추천<\/b>합니다,https://blog.naver.com/chycc07/221144662973,blog\n꼬부기네 여행 17개의 글\n꼬부기네 여행\n목록닫기\n꼬부기네 여행 글...,강원,횡성
82505,강원도 <b>횡성여행<\/b> 숨은맛집 허대감네막국수묵밥 <b>추천<\/b>,https://blog.naver.com/wawaw0116/220860446378,행복한인생 일상\n행복한인생 일상 319개의 글\n행복한인생 일상\n목록열기\n강원...,강원,횡성


In [36]:
text = data[['내용']].iloc[60000:82513] #범위 설정
text

,내용
60000,2022.05.22\n1박 2일 서울 친구네 여행의 마무리는\n#은평한옥마을 그리고...
60001,"은평구청에서 여권을 수령하러 가기 전, 여권 온라인 발급을 미리 해놨었거든, 그런데..."
60002,은평 한옥마을에 박물관도 있고\n근처에 맛집과 카페도 있다길래\n주말에 아기와 시간...
60003,구파발의 유래를 아시나요?\n글/사진 : 김미드\n주차가능 : 주차요금 있음 2시간...
60004,첫째가 태어나던 해에 개봉했던 아바타이다. 13년 만에 2편 물의 길을 공개했다. ...
...,...
82502,강\n원도 횡성에서 유명한 것이 바로 '더덕'이죠. 횡성더덕으로 코스요리를 전문으로...
82503,횡성 호텔#호텔 타임 #특실 #횡성 버스터미널 #횡성 호텔 추천 #금연 호텔\n강원...
82504,blog\n꼬부기네 여행 17개의 글\n꼬부기네 여행\n목록닫기\n꼬부기네 여행 글...
82505,행복한인생 일상\n행복한인생 일상 319개의 글\n행복한인생 일상\n목록열기\n강원...


### 명사 토큰화

In [37]:
text['내용'] = text.apply(lambda row: okt.nouns(row['내용']) if isinstance(row['내용'], str) else [], axis=1)

In [38]:
text

,내용
60000,"[박, 서울, 친구, 여행, 마무리, 먼저, 은평, 옥, 마을, 처음, 마침, 곳곳..."
60001,"[은평, 구청, 여권, 수령, 가기, 전, 여권, 온라인, 발급, 미리, 여권사진,..."
60002,"[은평, 옥, 마을, 박물관, 근처, 맛집, 카페, 주말, 아기, 시간, 개월, 아..."
60003,"[구파발, 유래, 글, 사진, 김, 미드, 주차, 가능, 주차, 요금, 이내, 관람..."
60004,"[첫째, 해, 개봉, 아바타, 편, 물의, 길, 공개, 첫째, 영화, 자기, 취향,..."
...,...
82502,"[강, 원도, 횡성, 것, 바로, 더덕, 이, 횡성, 더덕, 코스, 요리, 전문, ..."
82503,"[횡성, 호텔, 호텔, 타임, 버스, 터미널, 호텔, 추천, 호텔, 강원도, 횡성,..."
82504,"[꼬부기, 여행, 개, 글, 꼬부기, 여행, 목록, 꼬부기, 여행, 글, 목록, 글..."
82505,"[인생, 일상, 인생, 일상, 개, 글, 인생, 일상, 목록, 열기, 강원도, 횡성..."


In [40]:
text.to_csv("60000~82513국내여행지_토큰화.csv", encoding="utf-8", index=False)